<h1> AO* Search Algorithm</h1>

<h2>Pseudocode</h2>

<pre>
Initialization:
    Set up initial graph with heuristic costs
    Set terminal nodes as solved.
    Set up and_edges
    Set MAX(FUTILITY) = 1000

Call aostar on head node
Repeat until head is solved or number of iterations >MAX{
    aostarUtil(head)
}

Function aostarUtil
    #aostarUtil
    if head is not visited{
        #explore_head
        For each and-arc or or-arc in from head{
            if or-arc{
            cost = h(node in arc) + EDGE cost
            } else if and-arc{}
            calculate cumulative cost of nodes from arc + EDGE costs
            }
        update head cost as minimum cost among the and-arc's and or-arc's
        If head is solved, track the path 
        }   
    }else 
    {
        #find_best_move
        find the best and-arc or or-arc from head node
        Call aostarUtil recursively for each node in best and-arc or from node in or-arc
        
        #check update
        Propagate the change of cost in the head node
        
    }
</pre>

<a href = 'https://drive.google.com/file/d/1RPXanmRIWqcnWTfZaVZQgg2LikY9s9De/view?usp=sharing'> heuristic map</a>

In [66]:
class Node:
    def __init__(self, index,cost,visited=False,is_solved=False, and_map=False, or_map=False):
        self.index=index
        self.cost=cost

        self.visited=visited
        self.is_solved=is_solved
        
        self.and_map= and_map
        self.or_map = or_map

        self.children=None
        self.path = False
        
    def __str__(self):
        return f'{self.index}:{self.cost}'
    
    def set_children(self,ch):
        self.children=ch
        

In [67]:
#Use case 1
node=[]
n_nodes = 10
#heuristic costs
cost=[None,0,6,12,9,5,7,1,4,4,1]
children = {1:[2,3,4], 2:[5,6], 3:[7], 4:[8,9], 5:[10]}

node = [Node(i, cost[i]) for i in range(n_nodes+1)] # not considering 0 to be a node? 

for p,ch in children.items():
    ch_nodes = [node[c] for c in ch] # converting child nodes to Node class
    node[p].set_children(ch_nodes)

and_edges=[]

ae = [node[1],(node[2],node[3])]
and_edges.append(ae)
node[2].and_map=True; node[3].and_map = True

ae = [node[4],(node[8],node[9])]
and_edges.append(ae)
node[8].and_map=True; node[9].and_map = True

for a in node:
    if not a.children: a.is_solved=True #set for all terminal nodes
    if a.and_map==False: a.or_map=True #if not and arc then it is or arc

In [68]:
#  #use case 2
# adj=[]

# n_nodes = 21
# #heuristic costs
# cost=[None,0,40,2,4,1,2,3,50,60,70,80,4,5,8,9,6,7,90,90,90,90]
# children = {1:[2,3,4], 2:[5,6,7], 3:[8,9], 4:[10,11], 5:[12,13], 6:[14,15], 
#            7:[16,17], 8:[18], 9:[19], 10:[20], 11:[21] }
# #Note: the first node is dummy node
# adj = [Node(i, cost[i]) for i in range(n_nodes+1)]
# for p,ch in children.items():
#     ch_nodes = [adj[c] for c in ch]
#     adj[p].set_children(ch_nodes)

# and_edges=[]

# and_edges.append([adj[1], (adj[3], adj[4])])
# adj[3].and_map = adj[4].and_map = True

# and_edges.append([adj[2],(adj[5],adj[6], adj[7])])
# adj[5].and_map = adj[6].and_map = adj[7].and_map=True

# for a in adj:
#     if not a.children: a.is_solved=True
#     if a.and_map==False: a.or_map=True
#     #print(f'{a.index} and {a.and_map} or {a.or_map} solved {a.is_solved}')

In [69]:
#function to get all sibling nodes of a node part of an and-arc
def get_andarc(c):
    for parent, child_tuple in and_edges:
        if c in child_tuple : return child_tuple

In [70]:
def explore_head(head):
    print(f'Head: {head.index}, Cost: {head.cost}')
    head.path=True
    head.visited=True; temp_cost = MAX; temp_map=[] # here temp_map is useless, but it'll be useful in the next funcs. defining for uniformity 
    #check if head has any and edges
    for c in head.children:
        print(f'Child of head -> {c}, part of and arc -> {c.and_map} / or arc -> {c.or_map} & temp cost -> {temp_cost} ')
        if c in temp_map: continue;
        if c.and_map: # if the child is in the and edge
            ae = get_andarc(c) 
            cc=sum(aek.cost+EDGE for aek in ae)
            and_arc_solved = all([aek.is_solved for aek in ae]) # if all of them are leaf nodes, then the immediate parent = solved (coz children already explored)
            # not really right because, what if the node has an and arc + or arc -> but who cares lololol oh wait, they are checking here only for all children 
            # so in the next else, part they're checking for or arcs, so this shouldn't be a problemo
            if and_arc_solved: 
                head.is_solved=True;
                if cc<temp_cost: 
                    bestsolved=ae;
            print(f'Cost -> {cc}, solved = {and_arc_solved}')
            temp_cost = min(temp_cost,cc)

        else: # else if child is in the or edge
            cc = c.cost+EDGE
            if c.is_solved: # if or arc is solved
                head.is_solved=True 
                if cc < temp_cost: 
                    bestsolved = [c]
            temp_cost = min(temp_cost,cc)
            print(f'Cost -> {cc}, solved = {c.is_solved}')
   #head is explored now update the best value of head i.e. temp_cost  
    if temp_cost < MAX:
        head.cost=temp_cost
        print(f'Updated head {head.index} cost {head.cost} is_solved {head.is_solved}')
 
    #after all the children are explored & solved, the head is obviously solved
    if head.is_solved:
        for b in bestsolved: b.path=True
        for c in head.children: 
            if c not in bestsolved: c.path=False # i.e that child is not a part of best path, used while printing best path
    

In [71]:
# to fin
def find_best_move(head):
     #find the best move
        print(f'Find best move in node {head.index}')
        bestCost=MAX;bestMove=None; bestAndArc=None
        temp_map=[]
        for c in head.children:
            if c in temp_map:
                continue
            if c.or_map:
                cc = c.cost+EDGE
                if bestCost>cc:
                    bestCost = cc
                    bestMove=c; 
            else: # and map
                ae = get_andarc(c)
                temp_map.extend(ae) # adding elements to the list, 
                cc=sum(aek.cost+EDGE for aek in ae)
                if bestCost>cc and cc!=0:
                    bestCost = cc; bestAndArc = ae; bestMove = c
            
        print(f'\n Moving forward, finding the best move,i>> {bestMove.index} and_map {bestMove.and_map} cost: {bestCost}')
        if bestMove.and_map:
            for a in bestAndArc: # bestAndArc is a list, iterating through it to get individual costs 
                aostarUtil(a); 
        else:
            aostarUtil(bestMove); 


In [72]:
def check_update(head):
    temp_cost=MAX; temp_map=[]
    print(f'---------------Check update {head.index}----------------------------')
    for c in head.children:
        if c in temp_map:
            continue
        if c.or_map:
            if c.is_solved: 
                head.is_solved=True
            cc = c.cost+EDGE
            temp_cost= min(temp_cost, cc)
        elif c.and_map:
            ae = get_andarc(c)
            temp_map.extend(ae)
            temp_solved = all([aek.is_solved for aek in ae])
            cc=sum(aek.cost+EDGE for aek in ae)
            temp_cost = min(temp_cost,cc)
            
            if temp_solved : 
                head.is_solved = True

    if temp_cost<=MAX: 
        head.cost = temp_cost
    print(f'Propagate updated Cost of node {head.index} {head.cost} ')



In [73]:
path=[]
def aostarUtil(head):
    print(f'--------------------------------------aostarUtil on {head.index} which is visited -> {head.visited}---------------------------------------')
    if head.visited ==False:
        explore_head(head)  
    else:
        find_best_move(head)
        #check if any of the options were solved
        #if there are multiple solved nodes , select the best out of them 
        #also update the current cost i.e. head cost while backtracking to the root 
        check_update(head)

In [77]:
def aostar(head):
    iter = 0
    while head.is_solved==False and iter <MAX:
        print(f'\n\nIteration {iter}')
        aostarUtil(head)
        iter+=1
    
    print('\n Updated costs')
    for a in node[1:]:
        print(a.index,':',a.cost, a.is_solved)

In [78]:
MAX=1000
EDGE=1 #g cost of edge, change to 5 for use case 2
aostar(node[1]) 


 Updated costs
1 : 11 True
2 : 6 False
3 : 12 False
4 : 10 True
5 : 5 False
6 : 7 True
7 : 1 True
8 : 4 True
9 : 4 True
10 : 1 True


In [79]:
print(f'Cost of Solution : {node[1].cost}')
print('Path')
for a in node:
    if a.path: print(f'{a.index} cost: {a.cost}')

Cost of Solution : 11
Path
1 cost: 11
4 cost: 10
8 cost: 4
9 cost: 4
